In [2]:
import scipy.stats as st
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [3]:
np.random.seed(36)

In [4]:
class DatasetGenerator(object):
    def __init__(self, N=100):
        self.N = N
        self.x = None
        self.v = None
        self.refresh()
  
    def refresh(self):
        raise Exception("undefined")

In [5]:
class G1(DatasetGenerator):
    def refresh(self):
        self.x = st.uniform().rvs((self.N,2))
        self.v = st.uniform().rvs((self.N,))

In [6]:
class G2(DatasetGenerator):
    def refresh(self):
        self.x = st.uniform().rvs((self.N,2))
        self.v = np.exp(st.norm(-0.85, 1.3).rvs((self.N,)))

# Justin's notebook

## Computing iteratively the max distance between cities

The state is represented by the vector $\underline{x} := (x_1, x_2, ..., x_n) $, where $x_i \in \{0, 1\}$. If $x_i = 1$, it means that city $v_i$ is in our current set $\mathcal{S}$.

The Metropolis-Hasting chain evolves by adding or removing cities by proposing moves that flips exactly one entry of the state.

Computing the minimal distance in the set of cities $\mathcal{S}_{\underline{x}}$ associated with state $\underline{x}$ naively is not an option. Indeed, there are $2^n$ states and for each of them it would have a $\mathcal{O}(|\mathcal{S}_{\underline{x}}|)$ complexity.

The solution is to proceed iteratively. Starting from $\mathcal{S}_{\underline{0}} = (0, 0, ..., 0)$, we know that there are no elements in it, so the minial distance is zero.

Assume now that we are at a state, if we add a new city, it can be either in the circle, or outside. If it is inside, the distance does not increase, otherwise, the radius is the max of the distance of the new point to the two "border" points (can prove using Triangle inequality I guess). If we remove a state, we either remove an inner points (min distance does not decrease), or one of the two border points. In that case, we can again show that the other border point stays a border point (by contradiction) and we just have to compute the distance with respect to all other remaining points, which is done in $\mathcal{O}(n)$ complexity. 

This way, I think we can simulate the chain, and compute acceptances probabilities on the go (i. e. during simulation) and simulate easily the chain. 

The remaining issue, is how to tune the $\beta$ parameter of the Ising distribution, for that we may need to lookup on internet and try some things.

In [7]:
def max_squared_dist(points):
    p1 = np.repeat(points, len(points), axis = 0).reshape((len(points), len(points), 2), order = "F")
    p2 = np.repeat(points, len(points), axis = 0).reshape((len(points), len(points), 2), order = "C")
    return np.max(np.sum(np.square(p1 - p2), axis = 2))

def f(_lambda, x, v):
    return np.sum(v) - _lambda * len(v) * np.pi * max_squared_dist(x) / 4

In [8]:
def max_squared_dist2(points):
    N = len(points)
    p1 = points.reshape(1, N, 2)
    p2 = points.reshape(N, 1, 2)
    return np.max(np.sum(np.square(p1 - p2), axis = 2))

In [24]:
a = np.random.randint(-30, 30, size=(5000, 2))
print(max_squared_dist(a))
print(max_squared_dist2(a))

6962
6962


In [25]:
%%timeit
max_squared_dist(a)

3.97 s ± 891 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit
max_squared_dist2(a)

1.22 s ± 57.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
